In [1]:
import cv2
from ultralytics import YOLO, solutions

model_bottle = YOLO("yolov8n.pt")

print(model_bottle.names)

cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# offsets 
wo = w/4
ho = h/20

# Define region points
region_points = [(w-wo, h-ho),(wo, h-ho), (wo, ho), (w-wo, ho)]

class_bottle = 39

classes_to_count = [class_bottle] 

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=region_points,
    classes_names=model_bottle.names,
    draw_tracks=True,
    line_thickness=2,
)

bottle_count = 0
tracked = []
last_count = 0
counted = 0

# Video writer
video_writer = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model_bottle.track(im0, persist=True, show=False, classes=classes_to_count)

    im0 = counter.start_counting(im0, tracks)
    
    for i, r in enumerate(tracks):
        for index, box in enumerate(r.boxes):
            tracker_id = box.id

    for obj in tracks:
        if counter.out_counts+counter.in_counts > last_count:
            if tracker_id not in tracked:
                if obj.boxes[0].cls == class_bottle:  
                    bottle_count += 1
                    tracked.append(tracker_id)

    print("Bottles counted: ", bottle_count)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

# Print number of bottles counted
print("=====================================")
print("Total of bottles counted: ", bottle_count)


100%|██████████| 6.23M/6.23M [00:01<00:00, 5.68MB/s]


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw